In [5]:
import json
from rule_processing_functions import *

In [4]:
bool(1), bool('0')

(True, True)

## Creating Rule Templates for Rule Learning

In [102]:
def generate_rule_templates(predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, limed_Cultivated_DepthCulti, count = 0):
    new_rules = []
    descriptions = []
    
    #C_predicate = 'fs-data:Total-Organic-Carbon___LP_Heanes_RP____PC___PSC|fs-data:Total-Organic-Carbon___LP_Walkley-Black_RP____PC___PSC|fs-data:Total-Carbon___LP_Combustion_RP____PC___PSC'
    C_predicate = 'fs-data:Total-Carbon-_LP_Any-MoA_RP_-_PC___PSC'
    Al_predicate = 'fs-data:Aluminium-_PC_-of-Cations___LP_Calculated_RP____PC___PSC'
    ECEC_predicate = 'fs-data:Effective-Cation-Exchange-Capacity___LP_Calculated_RP___cmol_LP__Pl__RP__Pr_kg__PSC'


    #count = 0
    for t1, t2 in how_late:
        select_bind_filter_string = f'SELECT[]\tBIND[]\tFILTER[(?m_X - ?m_lime >= {t1} && ?m_X - ?m_lime <= {t2}), (?m_pH - ?m_lime <= {lime_observation_gap} && ?m_lime - ?m_pH <= {lime_observation_gap})]\t'
        head_string = f'{predictor}(?X,?pH_bin_X) <= fs-onto:hasObservation(?eccmo_X,?X), sosa:observedProperty(?X,fs-data:ObservedProperty-pH), sosa:usedProcedure(?X,fs-data:MoA-1-5-CaCl2), fs-onto:hasDepth(?X, ?d_pH), sosa:phenomenonTime(?X, ?t_X), fs-onto:hasMonthCount(?t_X, ?m_X), '

        for val in link_through_EccmoByRep:
            observations_at_Liming = ''
            if val==True:
                link_control = f'fs-onto:hasECCMO(?eccmoByRep_X, ?eccmo_X), rdf:type(?eccmoByRep_X, fs-onto:ECCMObyRep), fs-onto:hasControl(?eccmoByRep_X, ?eccmo_C), '
                link_eccmo = '?eccmo_C'
            else: 
                link_control = ''
                link_eccmo = '?eccmo_X'
            pH_at_Liming = f'fs-onto:hasObservation({link_eccmo}, ?obs_pH), fs-data:pH___LP_1-5-CaCl2_RP___PSC(?obs_pH,?pH_bin), fs-onto:hasDepth(?obs_pH, ?d_pH), sosa:phenomenonTime(?obs_pH, ?t_pH), fs-onto:hasMonthCount(?t_pH, ?m_pH), '

            for other_obs in additional_Observations_at_Liming:
                observations_at_Liming = ''
                if 'Carbon' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_C), sosa:phenomenonTime(?obs_C, ?t_pH),  fs-onto:hasDepth(?obs_C, ?d_pH), {C_predicate}(?obs_C, ?C_bin), '
                if 'Al' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_Al), sosa:phenomenonTime(?obs_Al, ?t_pH), fs-onto:hasDepth(?obs_Al, ?d_pH),  {Al_predicate}(?obs_Al, ?Al_bin), '
                if 'ECEC' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_ECEC), sosa:phenomenonTime(?obs_ECEC, ?t_pH), fs-onto:hasDepth(?obs_ECEC, ?d_pH),  {ECEC_predicate}(?obs_ECEC, ?ECEC_bin), '


                for l,c,d in limed_Cultivated_DepthCulti:
                    #print()
                    short_desc = f'i:{count},\t t1-t2: {t1}-{t2},\t link control: {val},\t other obs: {other_obs},\t limed: {l},\t cultivated: {c},\t depth of cultivation: {d}'
                    #print(short_desc)
                    descriptions.append(short_desc)
                    lime_var = '?lr_bin'
                    culti_var = '?culti_lvl'
                    d_culti_var = '?d_culti_bin'

                    if l=='any' or l=='no': #'any'|'bound'|'no'|'-'
                        lime_var = '?lr_bin_any'
                    if c=='any' or c=='no':
                        culti_var = '?culti_lvl_any'
                    if d=='any' or d=='no':
                         d_culti_var = '?d_culti_bin_any'

                    lime_str = f'fs-onto:hasECCMO(?mu_lime, ?eccmo_X), fs-onto:hasTime(?mu_lime, ?t_lime), fs-onto:hasMonthCount(?t_lime, ?m_lime), fs-onto:hasManagement(?mu_lime,?mng_lime), rdf:type(?mng_lime, fs-onto:Management-Amelioration-Lime), fs-data:Lime-Application-Rate__PSC(?mng_lime, {lime_var}), '
                    culti_str = f'fs-onto:hasECCMO(?mu_culti, ?eccmo_X), fs-onto:hasTime(?mu_culti, ?t_culti), fs-onto:hasMonthCount(?t_culti, ?m_lime), fs-onto:hasManagement(?mu_culti,?mng_culti), rdf:type(?mng_culti, fs-onto:Management-Amelioration), fs-onto:hasCultivationTreatment(?mng_culti, ?culti), skos:broader(?culti, {culti_var}), '
                    d_culti_str = f'fs-data:Depth-Of-Cultivation__PSC(?mng_culti, {d_culti_var})'

                    culti_str_short = f'fs-onto:hasECCMO(?mu_culti, ?eccmo_X), fs-onto:hasTime(?mu_culti, ?t_culti), fs-onto:hasMonthCount(?t_culti, ?m_lime), fs-onto:hasManagement(?mu_culti,?mng_culti), rdf:type(?mng_culti, fs-onto:Management-Amelioration)'

                    if c=='no':
                        culti_str = '~{'+ culti_str_short + '}, '
                    elif c=='-':
                        culti_str = ''
                        lime_str = lime_str[:-2] #remove the ', ' at the end

                    if d=='no':
                        d_culti_str = '~{'+ d_culti_str + '}'
                    elif d=='-':
                        d_culti_str = ''
                        culti_str = culti_str[:-2] #remove the ', ' at the end

                    rule_string = select_bind_filter_string+head_string+link_control+pH_at_Liming+observations_at_Liming+lime_str+culti_str+d_culti_str

                    count += 1
                    new_rules.append(rule_string)

    return new_rules, descriptions

In [103]:
with open("./lime_rules_base_templates.json", "r") as f:
    json_data = json.load(f)
print(f'dictionary keys: {json_data.keys()}')
rules = json_data['rules']
combinations = json_data['combinations']
print(f'No. of rule templates loaded: {len(rules)}')

dictionary keys: dict_keys(['rules', 'combinations'])
No. of rule templates loaded: 8


In [104]:
predictor =  "fs-data:pH___LP_1-5-CaCl2_RP___PSC"
how_late = [(10,14), (22,26), (34,38)] #t1, t2
lime_observation_gap = 2 #t3
link_through_EccmoByRep = [False, True]
observations_at_Liming = 'pH'
additional_Observations_at_Liming = [(), ('Carbon'),('Al'),('ECEC')]#, ('Carbon', 'Al'), ('Carbon', 'ECEC'), ('Al', 'ECEC')]#, ('Carbon', 'Al', 'ECEC')]

# any: match with any value
# bound: to be replaced with a literal value during rule generation
# -: disregard
limed_Cultivated_DepthCulti = [('any','-','-'), ('bound','-','-'), ('any','no','-'), ('bound','no','-'),
							   ('any','any','-'), ('bound','bound','-'),							  
                               ('any','any','any'), #('any','any','no'),
							   #('bound','bound','bound'),  (removed)
                               ('bound','bound','any')]#, ('bound','bound','no')]


In [105]:
new_rules1, descriptions1 = generate_rule_templates(predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, limed_Cultivated_DepthCulti, count=0)

In [106]:
additional_Observations_at_Liming = [('Carbon', 'Al'), ('Carbon', 'ECEC'), ('Al', 'ECEC')] 
new_rules2, descriptions2 = generate_rule_templates(predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, limed_Cultivated_DepthCulti, count=len(new_rules1))

In [107]:
len(new_rules1), len(new_rules2), len(descriptions1), len(descriptions2)

(192, 144, 192, 144)

In [108]:
len(new_rules1+new_rules2)

336

In [109]:
rules = json_data['rules']
combinations = json_data['combinations']
#rules.extend(new_rules)

In [110]:
with open("./templates/lime_rules_templates.json", "w") as f:
    json.dump({'rules':new_rules1+new_rules2, 'combinations':combinations}, f)     

In [111]:
with open("./templates/lime_rules_templates.txt", "w") as f:
    for line in new_rules1+new_rules2:
        f.writelines(line+'\n')

In [113]:
with open("./templates/lime_rules_templates_descriptions.txt", "w") as f:
    for line in descriptions1+descriptions2:
        f.writelines(line+'\n')

## Create Attribute Extraction Rules

In [6]:
def generate_attibute_rules(att_predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, match_depth=True, count=0):
    new_att_rules = []
    descriptions = []
    
    #count = 0
    for t1, t2 in how_late:
        if match_depth:
            select_bind_filter_string = f'SELECT[((?d_pH_to + ?d_pH_from)/2 AS ?d_pH_avg), (?m_X - ?m_lime AS ?gap)]\tBIND[]\tFILTER[(?m_X - ?m_lime >= {t1} && ?m_X - ?m_lime <= {t2}), (?m_pH - ?m_lime <= {lime_observation_gap} && ?m_lime - ?m_pH <= {lime_observation_gap})]\t'
            head_string = f'{att_predictor}(?X,?pH_val_X) <= fs-onto:hasObservation(?eccmo_X,?X), sosa:observedProperty(?X,fs-data:ObservedProperty-pH), sosa:usedProcedure(?X,fs-data:MoA-1-5-CaCl2), fs-onto:hasDepth(?X, ?d_pH), sosa:phenomenonTime(?X, ?t_X), fs-onto:hasMonthCount(?t_X, ?m_X), '
        else: # get depth of X and to/from values separately
            select_bind_filter_string = f'SELECT[((?d_X_to + ?d_X_from)/2 AS ?d_X_avg), ((?d_pH_to + ?d_pH_from)/2 AS ?d_pH_avg), (?m_X - ?m_lime AS ?gap)]\tBIND[]\tFILTER[(?m_X - ?m_lime >= {t1} && ?m_X - ?m_lime <= {t2}), (?m_pH - ?m_lime <= {lime_observation_gap} && ?m_lime - ?m_pH <= {lime_observation_gap})]\t'
            head_string = f'{att_predictor}(?X,?pH_val_X) <= fs-onto:hasObservation(?eccmo_X,?X), sosa:observedProperty(?X,fs-data:ObservedProperty-pH), sosa:usedProcedure(?X,fs-data:MoA-1-5-CaCl2), fs-onto:hasDepth(?X, ?d_X), fs-onto:from(?d_X, ?d_X_from), fs-onto:to(?d_X, ?d_X_to), sosa:phenomenonTime(?X, ?t_X), fs-onto:hasMonthCount(?t_X, ?m_X), '
        for val in link_through_EccmoByRep:
            observations_at_Liming = ''
            if val==True:
                link_control = f'fs-onto:hasECCMO(?eccmoByRep_X, ?eccmo_X), rdf:type(?eccmoByRep_X, fs-onto:ECCMObyRep), fs-onto:hasControl(?eccmoByRep_X, ?eccmo_C), '
                link_eccmo = '?eccmo_C'
            else: 
                link_control = ''
                link_eccmo = '?eccmo_X'
            pH_at_Liming = f'fs-onto:hasObservation({link_eccmo}, ?obs_pH), fs-data:pH___LP_1-5-CaCl2_RP___ASC(?obs_pH,?pH_val), fs-onto:hasDepth(?obs_pH, ?d_pH), fs-onto:from(?d_pH, ?d_pH_from), fs-onto:to(?d_pH, ?d_pH_to), sosa:phenomenonTime(?obs_pH, ?t_pH), fs-onto:hasMonthCount(?t_pH, ?m_pH), '

            for other_obs in additional_Observations_at_Liming:
                observations_at_Liming = ''
                if 'Carbon' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_C), sosa:phenomenonTime(?obs_C, ?t_pH),  fs-onto:hasDepth(?obs_C, ?d_pH), {C_val_predicate}(?obs_C, ?C_val), '
                if 'Al' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_Al), sosa:phenomenonTime(?obs_Al, ?t_pH), fs-onto:hasDepth(?obs_Al, ?d_pH),  {Al_val_predicate}(?obs_Al, ?Al_val), '
                if 'ECEC' in other_obs:
                    observations_at_Liming += f'fs-onto:hasObservation({link_eccmo}, ?obs_ECEC), sosa:phenomenonTime(?obs_ECEC, ?t_pH), fs-onto:hasDepth(?obs_ECEC, ?d_pH),  {ECEC_val_predicate}(?obs_ECEC, ?ECEC_val), '

                short_desc = f'i:{count},\t t1-t2: {t1}-{t2},\t link control: {val},\t match depth: {match_depth},\t other obs: {other_obs}'
                #print(short_desc)
                descriptions.append(short_desc)
                
                lime_var = '?lr_val'
                culti_var = '?culti_lvl'
                d_culti_var = '?d_culti_val'

                lime_str = f'fs-onto:hasECCMO(?mu_lime, ?eccmo_X), fs-onto:hasTime(?mu_lime, ?t_lime), fs-onto:hasMonthCount(?t_lime, ?m_lime), fs-onto:hasManagement(?mu_lime,?mng_lime), rdf:type(?mng_lime, fs-onto:Management-Amelioration-Lime), fs-onto:hasLimeApplicationRate(?mng_lime, {lime_var}), '
                d_lime_str = f'{{fs-onto:hasDepthOfLimePlacement(?mng_lime, ?d_lime)}}, '
                culti_str = f'{{fs-onto:hasECCMO(?mu_culti, ?eccmo_X), fs-onto:hasTime(?mu_culti, ?t_culti), fs-onto:hasMonthCount(?t_culti, ?m_lime), fs-onto:hasManagement(?mu_culti,?mng_culti), rdf:type(?mng_culti, fs-onto:Management-Amelioration), fs-onto:hasCultivationTreatment(?mng_culti, ?culti), skos:broader(?culti, {culti_var}), fs-onto:index({culti_var}, ?culti_idx), '
                d_culti_str = f'{{fs-onto:hasDepthOfCultivation(?mng_culti, {d_culti_var})}}}}'

                rule_string = select_bind_filter_string+head_string+link_control+pH_at_Liming+observations_at_Liming+lime_str+d_lime_str+culti_str+d_culti_str
                #print(rule_string)
                #select_exp, bind_exp, filter_exp, rule_exp = rule_string.split('\t')
                #rule = re.sub(r'\t|\n|<=|\(|\)|\{|\}|,|\~', ' ', rule_exp).split()
                #bodys = np.array(rule[3:]).reshape(int(len(rule[3:]) / 3), 3)
                #rs, bs, hs, numerator_query, denominator_query, denominator_query_HC = compile_query_for_rule_v2(rule_string, optional_clause=True, verbrose=False)
                #res = data.run_query_df(rs)
                count += 1
                new_att_rules.append(rule_string)
                
    return new_att_rules, descriptions



In [7]:

att_predictor =  "fs-data:pH___LP_1-5-CaCl2_RP___ASC"
how_late = [(6,60)] #[(10,14), (22,26), (34,38), (6,60)] #t1, t2
lime_observation_gap = 2 #t3
link_through_EccmoByRep = [False, True]
observations_at_Liming = 'pH'
additional_Observations_at_Liming = [(), ('Carbon'),('Al'),('ECEC'), ('Carbon', 'Al'), ('Carbon', 'ECEC'), ('Al', 'ECEC')]#, ('Carbon', 'Al', 'ECEC')]

#C_val_predicate = 'fs-data:Total-Organic-Carbon___LP_Heanes_RP____PC___ASC|fs-data:Total-Organic-Carbon___LP_Walkley-Black_RP____PC___ASC|fs-data:Total-Carbon___LP_Combustion_RP____PC___ASC'
C_val_predicate = 'fs-data:Total-Carbon-_LP_Any-MoA_RP_-_PC___ASC'
Al_val_predicate = 'fs-data:Aluminium-_PC_-of-Cations___LP_Calculated_RP____PC___ASC'
ECEC_val_predicate = 'fs-data:Effective-Cation-Exchange-Capacity___LP_Calculated_RP___cmol_LP__Pl__RP__Pr_kg__ASC'

match_depth=True  # whether to get observations at the same depth interval as the Observation (or else, gather soil observations at different depths)

In [8]:
new_att_rules1, att_rule_descriptions1 = generate_attibute_rules(att_predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, match_depth, count = 0)

In [9]:
match_depth=False
new_att_rules2, att_rule_descriptions2 = generate_attibute_rules(att_predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, match_depth, count = len(new_att_rules1))

In [16]:
how_late = [(10,14), (22,26), (34,38)] # 1 yr later, 2 yrs later, 3 yrs later
match_depth=True
new_att_rules3, att_rule_descriptions3 = generate_attibute_rules(att_predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, match_depth, count = (len(new_att_rules1)+len(new_att_rules2)))

match_depth=False
new_att_rules4, att_rule_descriptions4 = generate_attibute_rules(att_predictor, how_late, lime_observation_gap, link_through_EccmoByRep, observations_at_Liming, additional_Observations_at_Liming, match_depth, count = (len(new_att_rules1)+len(new_att_rules2)+len(new_att_rules3)))

In [17]:
len(new_att_rules1), len(att_rule_descriptions1), len(new_att_rules2), len(att_rule_descriptions2),  len(new_att_rules3), len(att_rule_descriptions3),  len(new_att_rules4), len(att_rule_descriptions4)

(14, 14, 14, 14, 42, 42, 42, 42)

In [18]:
with open("./templates/lime_attribute_rules.json", "w") as f:
    json.dump({'att_rules':new_att_rules1+new_att_rules2+new_att_rules3+new_att_rules4}, f) 

In [19]:
with open("./templates/lime_attribute_rules.txt", "w") as f:
    for line in new_att_rules1+new_att_rules2+new_att_rules3+new_att_rules4:
        f.writelines(line+'\n')

In [20]:
with open("./templates/lime_attribute_rules_descriptions.txt", "w") as f:
    for line in att_rule_descriptions1+att_rule_descriptions2+att_rule_descriptions3+att_rule_descriptions4:
        f.writelines(line+'\n')

## Testing Attribute extraction

In [115]:
from rule_processing_functions import *
from QueryExecutor import *
data = QueryExecutor()  

In [117]:
n = 7
print(new_att_rules2[n])
rs, bs, hs, numerator_query, denominator_query, denominator_query_HC = compile_query_for_rule_v2(new_att_rules2[n], optional_clause=True, verbrose=False)
#res_rs = data.run_query_df(rs)
#print("rs", len(res_rs))

print()
print(rs)
res_rs = data.run_query_df(rs)

print("\nrs", len(res_rs))
res_rs

SELECT[((?d_X_to + ?d_X_from)/2 AS ?d_X_avg), ((?d_pH_to + ?d_pH_from)/2 AS ?d_pH_avg), (?m_X - ?m_lime AS ?gap)]	BIND[]	FILTER[(?m_X - ?m_lime >= 6 && ?m_X - ?m_lime <= 60), (?m_pH - ?m_lime <= 2 && ?m_lime - ?m_pH <= 2)]	fs-data:pH___LP_1-5-CaCl2_RP___ASC(?X,?pH_val_X) <= fs-onto:hasObservation(?eccmo_X,?X), sosa:observedProperty(?X,fs-data:ObservedProperty-pH), sosa:usedProcedure(?X,fs-data:MoA-1-5-CaCl2), fs-onto:hasDepth(?X, ?d_X), fs-onto:from(?d_X, ?d_X_from), fs-onto:to(?d_X, ?d_X_to), sosa:phenomenonTime(?X, ?t_X), fs-onto:hasMonthCount(?t_X, ?m_X), fs-onto:hasECCMO(?eccmoByRep_X, ?eccmo_X), rdf:type(?eccmoByRep_X, fs-onto:ECCMObyRep), fs-onto:hasControl(?eccmoByRep_X, ?eccmo_C), fs-onto:hasObservation(?eccmo_C, ?obs_pH), fs-data:pH___LP_1-5-CaCl2_RP___ASC(?obs_pH,?pH_val), fs-onto:hasDepth(?obs_pH, ?d_pH), fs-onto:from(?d_pH, ?d_pH_from), fs-onto:to(?d_pH, ?d_pH_to), sosa:phenomenonTime(?obs_pH, ?t_pH), fs-onto:hasMonthCount(?t_pH, ?m_pH), fs-onto:hasECCMO(?mu_lime, ?eccmo_X)

,eccmo_X,X,d_X,d_X_from,d_X_to,t_X,m_X,eccmoByRep_X,eccmo_C,obs_pH,...,t_culti,mng_culti,culti,culti_lvl,culti_idx,d_culti_val,d_X_avg,d_pH_avg,gap,pH_val_X
0,fs-data:ECCMO-023f8d2c73eace,fs-data:Observation-844714c3329e27-743-O,fs-data:Depth-7.5-10.0,7.5,10.0,fs-data:Time-2023-01-17,637.0,fs-data:ECCMObyRep-9a84c6899c8488,fs-data:ECCMO-56199cee3c9778,fs-data:Observation-844714c3329e27-012-O,...,fs-data:Time-2019-10-24,fs-data:Management-Amelioration-2e917e62d2ae63,fs-data:CultivationTreatment-DISC-HARROWS-inst...,fs-data:CultivationTreatmentLevel-L2-Medium_So...,2.0,10.0,8.75,2.50,39.0,5.2
1,fs-data:ECCMO-023f8d2c73eace,fs-data:Observation-844714c3329e27-127-O,fs-data:Depth-17.5-20.0,17.5,20.0,fs-data:Time-2020-12-14,612.0,fs-data:ECCMObyRep-9a84c6899c8488,fs-data:ECCMO-56199cee3c9778,fs-data:Observation-844714c3329e27-012-O,...,fs-data:Time-2019-10-24,fs-data:Management-Amelioration-2e917e62d2ae63,fs-data:CultivationTreatment-DISC-HARROWS-inst...,fs-data:CultivationTreatmentLevel-L2-Medium_So...,2.0,10.0,18.75,2.50,14.0,4.6
2,fs-data:ECCMO-023f8d2c73eace,fs-data:Observation-844714c3329e27-437-O,fs-data:Depth-17.5-20.0,17.5,20.0,fs-data:Time-2022-02-10,626.0,fs-data:ECCMObyRep-9a84c6899c8488,fs-data:ECCMO-56199cee3c9778,fs-data:Observation-844714c3329e27-012-O,...,fs-data:Time-2019-10-24,fs-data:Management-Amelioration-2e917e62d2ae63,fs-data:CultivationTreatment-DISC-HARROWS-inst...,fs-data:CultivationTreatmentLevel-L2-Medium_So...,2.0,10.0,18.75,2.50,28.0,4.8
3,fs-data:ECCMO-023f8d2c73eace,fs-data:Observation-844714c3329e27-749-O,fs-data:Depth-25.0-30.0,25.0,30.0,fs-data:Time-2023-01-17,637.0,fs-data:ECCMObyRep-9a84c6899c8488,fs-data:ECCMO-56199cee3c9778,fs-data:Observation-844714c3329e27-012-O,...,fs-data:Time-2019-10-24,fs-data:Management-Amelioration-2e917e62d2ae63,fs-data:CultivationTreatment-DISC-HARROWS-inst...,fs-data:CultivationTreatmentLevel-L2-Medium_So...,2.0,10.0,27.50,2.50,39.0,4.9
4,fs-data:ECCMO-023f8d2c73eace,fs-data:Observation-844714c3329e27-121-O,fs-data:Depth-2.5-5.0,2.5,5.0,fs-data:Time-2020-12-14,612.0,fs-data:ECCMObyRep-9a84c6899c8488,fs-data:ECCMO-56199cee3c9778,fs-data:Observation-844714c3329e27-012-O,...,fs-data:Time-2019-10-24,fs-data:Management-Amelioration-2e917e62d2ae63,fs-data:CultivationTreatment-DISC-HARROWS-inst...,fs-data:CultivationTreatmentLevel-L2-Medium_So...,2.0,10.0,3.75,2.50,14.0,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36704,fs-data:ECCMO-f2b15f63eaac67,fs-data:Observation-d7e84fde17c71a-146-O,fs-data:Depth-12.5-15.0,12.5,15.0,fs-data:Time-2020-12-07,612.0,fs-data:ECCMObyRep-cd3850ef23666f,fs-data:ECCMO-98556309454fd9,fs-data:Observation-8fa1a54adfc1ad-19-O,...,fs-data:Time-2020-02-06,fs-data:Management-Amelioration-7740cb648163bd,fs-data:CultivationTreatment-INCORPORATED-BY-S...,fs-data:CultivationTreatmentLevel-L1-Low_Soil_...,1.0,NaN,13.75,8.75,10.0,5.2
36705,fs-data:ECCMO-f2b15f63eaac67,fs-data:Observation-d7e84fde17c71a-142-O,fs-data:Depth-2.5-5.0,2.5,5.0,fs-data:Time-2020-12-07,612.0,fs-data:ECCMObyRep-cd3850ef23666f,fs-data:ECCMO-98556309454fd9,fs-data:Observation-8fa1a54adfc1ad-19-O,...,fs-data:Time-2020-02-06,fs-data:Management-Amelioration-7740cb648163bd,fs-data:CultivationTreatment-INCORPORATED-BY-S...,fs-data:CultivationTreatmentLevel-L1-Low_Soil_...,1.0,NaN,3.75,8.75,10.0,5.0
36706,fs-data:ECCMO-f2b15f63eaac67,fs-data:Observation-d7e84fde17c71a-150-O,fs-data:Depth-25.0-30.0,25.0,30.0,fs-data:Time-2020-12-07,612.0,fs-data:ECCMObyRep-cd3850ef23666f,fs-data:ECCMO-98556309454fd9,fs-data:Observation-8fa1a54adfc1ad-19-O,...,fs-data:Time-2020-02-06,fs-data:Management-Amelioration-7740cb648163bd,fs-data:CultivationTreatment-INCORPORATED-BY-S...,fs-data:CultivationTreatmentLevel-L1-Low_Soil_...,1.0,NaN,27.50,8.75,10.0,6.1
36707,fs-data:ECCMO-f2b15f63eaac67,fs-data:Observation-d7e84fde17c71a-144-O,fs-data:Depth-7.5-10.0,7.5,10.0,fs-data:Time-2020-12-07,612.0,fs-data:ECCMObyRep-cd3850ef23666f,fs-data:ECCMO-98556309454fd9,fs-data:O

In [118]:
res_rs.isna().sum()

eccmo_X             0
X                   0
d_X                 0
d_X_from            0
d_X_to              0
t_X                 0
m_X                 0
eccmoByRep_X        0
eccmo_C             0
obs_pH              0
pH_val              0
d_pH                0
d_pH_from           0
d_pH_to             0
t_pH                0
m_pH                0
mu_lime             0
t_lime              0
m_lime              0
mng_lime            0
lr_val              0
d_lime          36709
mu_culti         3024
t_culti          3024
mng_culti        3024
culti            3024
culti_lvl        3024
culti_idx        3024
d_culti_val     16794
d_X_avg             0
d_pH_avg            0
gap                 0
pH_val_X            0
dtype: int64

In [121]:
res_rs[['d_X', 'd_X_from', 'd_X_to', 'd_X_avg', 'd_pH', 'd_pH_from', 'd_pH_to', 'd_pH_avg', 't_X', 't_pH']]

,d_X,d_X_from,d_X_to,d_X_avg,d_pH,d_pH_from,d_pH_to,d_pH_avg,t_X,t_pH
0,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Depth-0.0-5.0,0.0,5.0,2.50,fs-data:Time-2023-01-17,fs-data:Time-2019-09-19
1,fs-data:Depth-17.5-20.0,17.5,20.0,18.75,fs-data:Depth-0.0-5.0,0.0,5.0,2.50,fs-data:Time-2020-12-14,fs-data:Time-2019-09-19
2,fs-data:Depth-17.5-20.0,17.5,20.0,18.75,fs-data:Depth-0.0-5.0,0.0,5.0,2.50,fs-data:Time-2022-02-10,fs-data:Time-2019-09-19
3,fs-data:Depth-25.0-30.0,25.0,30.0,27.50,fs-data:Depth-0.0-5.0,0.0,5.0,2.50,fs-data:Time-2023-01-17,fs-data:Time-2019-09-19
4,fs-data:Depth-2.5-5.0,2.5,5.0,3.75,fs-data:Depth-0.0-5.0,0.0,5.0,2.50,fs-data:Time-2020-12-14,fs-data:Time-2019-09-19
...,...,...,...,...,...,...,...,...,...,...
36704,fs-data:Depth-12.5-15.0,12.5,15.0,13.75,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Time-2020-12-07,fs-data:Time-2019-12-19
36705,fs-data:Depth-2.5-5.0,2.5,5.0,3.75,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Time-2020-12-07,fs-data:Time-2019-12-19
36706,fs-data:Depth-25.0-30.0,25.0,30.0,27.50,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Time-2020-12-07,fs-data:Time-2019-12-19
36707,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Depth-7.5-10.0,7.5,10.0,8.75,fs-data:Time-2020-12-07,fs-data:Time-2019-12-19
